In [2]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df


,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,10,18,2024-07-01 19:28:19
1,大安區,21,14,7,2024-07-01 19:22:53
2,大安區,16,10,6,2024-07-01 19:24:20
3,大安區,11,5,6,2024-07-01 19:26:20
4,大安區,16,1,15,2024-07-01 19:28:19
...,...,...,...,...,...
1424,臺大公館校區,30,0,29,2024-07-01 19:19:15
1425,臺大公館校區,20,1,19,2024-07-01 19:28:20
1426,臺大公館校區,24,10,13,2024-07-01 19:21:21
1427,臺大公館校區,40,25,6,2024-07-01 19:28:20


In [7]:
df1 = df.set_index(keys=["行政區域"])
df1.groupby(level='行政區域')[['總數量','可借數量','可還數量']].sum()

,總數量,可借數量,可還數量
行政區域,,,
中山區,3819,1620,2180
中正區,3209,1499,1681
信義區,3331,1374,1869
內湖區,3499,1432,2032
北投區,2472,1012,1433
南港區,2317,1156,1141
士林區,2937,1100,1777
大同區,1629,648,972
大安區,4987,2152,2758
